In [2]:
import pandas as pd
import numpy as np

## Load BP data

In [3]:
df = pd.read_csv('https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/xlsx/energy-economics/statistical-review/bp-stats-review-2022-consolidated-dataset-narrow-format.csv')
df.head()

,Country,Year,ISO3166_alpha3,ISO3166_numeric,Region,SubRegion,OPEC,EU,OECD,CIS,Var,Value
0,Brazil,2004,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,0.000000
1,Brazil,2005,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,0.014737
2,Brazil,2006,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,1.016887
3,Brazil,2007,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,5.953947
4,Brazil,2008,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,16.534379


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277095 entries, 0 to 277094
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Country          277095 non-null  object 
 1   Year             277095 non-null  int64  
 2   ISO3166_alpha3   277095 non-null  object 
 3   ISO3166_numeric  193275 non-null  float64
 4   Region           193275 non-null  object 
 5   SubRegion        193275 non-null  object 
 6   OPEC             193275 non-null  float64
 7   EU               193275 non-null  float64
 8   OECD             193275 non-null  float64
 9   CIS              193275 non-null  float64
 10  Var              277095 non-null  object 
 11  Value            277095 non-null  float64
dtypes: float64(6), int64(1), object(5)
memory usage: 25.4+ MB


In [5]:
df.Var.unique()

array(['biodiesel_cons_kboed', 'biodiesel_cons_pj',
       'biodiesel_prod_kboed', 'biodiesel_prod_pj', 'biofuels_cons_ej',
       'biofuels_cons_kbd', 'biofuels_cons_kboed', 'biofuels_cons_pj',
       'biofuels_prod_kbd', 'biofuels_prod_kboed', 'biofuels_prod_pj',
       'biogeo_ej', 'biogeo_twh', 'biogeo_twh_net', 'co2_combust_mtco2',
       'co2_combust_pc', 'co2_combust_per_ej', 'co2_mtco2', 'coalcons_ej',
       'coalprod_ej', 'coalprod_mt', 'cobalt_kt', 'cobaltres_kt',
       'diesel_gasoil_cons_kbd', 'elect_twh', 'electbyfuel_coal',
       'electbyfuel_gas', 'electbyfuel_hydro', 'electbyfuel_nuclear',
       'electbyfuel_oil', 'electbyfuel_other', 'electbyfuel_ren_power',
       'electbyfuel_total', 'ethanol_cons_kboed', 'ethanol_cons_pj',
       'ethanol_prod_kboed', 'ethanol_prod_pj', 'fuel_oil_cons_kbd',
       'gascons_bcfd', 'gascons_bcm', 'gascons_ej', 'gasflared_bcm',
       'gasflared_mtco2', 'gasoline_cons_kbd', 'gasprod_bcfd',
       'gasprod_bcm', 'gasprod_ej', 'gasre

# Proven Reserves

## Clean data

### Dimension Table

In [19]:
# Unit table 
data = [
    {'unit_id': 0, 'unit': 'bcm', 'energy_type': 'Gas', 'target': 'proven_reserves', 'conversion_coef': 1, 'group' : 0},
    {'unit_id': 1, 'unit': 'tcm', 'energy_type': 'Gas', 'target': 'proven_reserves', 'conversion_coef': 0.001, 'group' : 0},
    {'unit_id': 2, 'unit': 'gb', 'energy_type': 'Oil', 'target': 'proven_reserves', 'conversion_coef': 1, 'group' : 1},
    {'unit_id': 3, 'unit': 'bbl', 'energy_type': 'Oil', 'target': 'proven_reserves', 'conversion_coef': 1, 'group' : 1},
    {'unit_id': 4, 'unit': 'bcm', 'energy_type': 'Gas', 'target': 'production', 'conversion_coef': 1, 'group' : 2},
    {'unit_id': 5, 'unit': 'ej', 'energy_type': 'Gas', 'target': 'production', 'conversion_coef': 0.036, 'group' : 2},
    {'unit_id': 6, 'unit': 'bcfd', 'energy_type': 'Gas', 'target': 'production', 'conversion_coef': 0.0965, 'group' : 2},
    {'unit_id': 7, 'unit': 'mt', 'energy_type': 'Oil', 'target': 'production', 'conversion_coef': 1, 'group' : 3},
    {'unit_id': 8, 'unit': 'kbd', 'energy_type': 'Oil', 'target': 'production', 'conversion_coef': 1, 'group' : 4}
]

df_unit = pd.DataFrame(data)
# Create a mapping dictionary
unit_id_mapping = {}

for item in data:
    # Map based on unit, energy_type, target, and group
    key = (item['unit'], item['energy_type'], item['target'])
    unit_id = item['unit_id']
    unit_id_mapping[key] = unit_id
    
df_unit

,unit_id,unit,energy_type,target,conversion_coef,group
0,0,bcm,Gas,proven_reserves,1.0000,0
1,1,tcm,Gas,proven_reserves,0.0010,0
2,2,gb,Oil,proven_reserves,1.0000,1
3,3,bbl,Oil,proven_reserves,1.0000,1
4,4,bcm,Gas,production,1.0000,2
5,5,ej,Gas,production,0.0360,2
6,6,bcfd,Gas,production,0.0965,2
7,7,mt,Oil,production,1.0000,3
8,8,kbd,Oil,production,1.0000,4


### data_V0 : Data like in the previous shift data (pgadmin)

In [8]:
def clean_data_0(df):
    # Columns selection
    df_clean = df.drop(columns=['ISO3166_numeric', 'ISO3166_alpha3'], axis=1)
    
    # Filter oil & gas
    df_clean = df_clean[df_clean['Var'].isin(['gasreserves_tcm', 'oilreserves_bbl'])].reset_index(drop=True)
    
    # Add unit columns
    df_clean['proven_reserves_unit'] = np.where(df_clean['Var'] == 'gasreserves_tcm', 'tcm', np.where(df_clean['Var'] == 'oilreserves_bbl', 'bbl', ''))
    
    # Replace variable names
    df_clean['Var'].replace({'gasreserves_tcm': 'Gas', 'oilreserves_bbl': 'Oil'}, inplace=True)
    df_clean = df_clean.rename(columns={'Country': 'group_name', 'Year': 'year', 'Value': 'proven_reserves', 'Var': 'energy_source'})
    
    df_clean['group_type'] = 'country'
    df_clean['group_name'].replace({'Total OECD': 'OECD', 'Total OPEC': 'OPEC', 'Total EU': 'EU28', 'Total CIS': 'CIS'}, inplace=True)
    df_clean.loc[df_clean.group_name.isin(['OECD', 'OPEC', 'EU28', 'CIS']), 'group_type'] = 'group'
    
    df_clean['group_name'].replace({'Total S. & Cent. America': 'Central and South America', 'Total North America': 'North America', 'Total Africa': 'Africa', 'Total Europe': 'Europe', 'Total World': 'World', 'Total Middle East': 'Middle East', 'Total Asia Pacific': 'Asia Pacific'}, inplace=True)
    df_clean.loc[df_clean.group_name.isin(['Central and South America', 'North America', 'Africa', 'Europe', 'World', 'Middle East', 'Asia Pacific']), 'group_type'] = 'zone'
    
    df_clean = df_clean.drop(columns=['OPEC', 'EU', 'OECD', 'CIS', 'Region', 'SubRegion'], axis=1)
    df_clean['proven_reserves_unit'].replace({'bbl': 'Gb', 'tcm': 'Bcm'}, inplace=True)
    df_clean.loc[df_clean.proven_reserves_unit == 'Bcm', 'proven_reserves'] *= 1000
    
    return df_clean

In [9]:
data_v0 = clean_data_0(df)

In [12]:
data_v0.head()

,group_name,year,energy_source,proven_reserves,proven_reserves_unit,group_type
0,Algeria,1980,Gas,3581.4625,Bcm,country
1,Algeria,1981,Gas,3540.0750,Bcm,country
2,Algeria,1982,Gas,3477.5125,Bcm,country
3,Algeria,1983,Gas,3398.5875,Bcm,country
4,Algeria,1984,Gas,3312.9250,Bcm,country


### data_v1 : New data format but before using the unit dimension table

In [27]:
def clean_data_1(df):

    df =df.rename(columns={'ISO3166_alpha3': 'country_code_a3','Region' : 'region_name','Country':'country_name', 'Year' : 'year', 'Value': 'value', 'Var': 'type'})
    #filtering oil & gas 
    df = df[df['type'].isin(['gasreserves_tcm', 'oilreserves_bbl'])]
    #Deleting sums
    df = df[~df['country_code_a3'].str.contains('T-')]
    #Deleting world
    df = df[df['country_code_a3'] != 'WLD']
    #adding unit columns
    df['unit'] = np.where(df['type'] == 'gasreserves_tcm', 'tcm', np.where(df['type'] == 'oilreserves_bbl', 'bbl', ''))
    #filling region columns for "Other ..." countries
    mask = df['region_name'].str.startswith('Other ')
    df.loc[mask & df['region_name'].notna(), 'region_name'] = df.loc[mask & df['region_name'].notna(), 'country_name'].str.split('Other ').str[1]
    #Adding source 
    df['source']='bp'

    #Selecting columns 
    df_clean = df[['country_code_a3', 'country_name', 'region_name', 'type', 'year', 'value', 'unit','source']]

    return df_clean

In [28]:
data_V1 = clean_data_1(df)

In [29]:
data_V1.head()

,country_code_a3,country_name,region_name,type,year,value,unit,source
114381,DZA,Algeria,Africa,gasreserves_tcm,1980,3.581463,tcm,bp
114382,DZA,Algeria,Africa,gasreserves_tcm,1981,3.540075,tcm,bp
114383,DZA,Algeria,Africa,gasreserves_tcm,1982,3.477513,tcm,bp
114384,DZA,Algeria,Africa,gasreserves_tcm,1983,3.398588,tcm,bp
114385,DZA,Algeria,Africa,gasreserves_tcm,1984,3.312925,tcm,bp


### data_V2 : New data format  (final) 

In [17]:
def clean_data_2(df, unit_id_mapping):
    # Columns selection
    df_clean = df.drop(columns=['ISO3166_numeric'], axis=1)
    
    # Filter oil & gas
    df_clean = df_clean[df_clean['Var'].isin(['gasreserves_tcm', 'oilreserves_bbl'])].reset_index(drop=True)
    
    # Add unit columns
    df_clean['unit'] = np.where(df_clean['Var'] == 'gasreserves_tcm', 'tcm', np.where(df_clean['Var'] == 'oilreserves_bbl', 'bbl', ''))
    
    # Replace variable names
    df_clean['Var'].replace({'gasreserves_tcm': 'Gas', 'oilreserves_bbl': 'Oil'}, inplace=True)
    df_clean['target'] = 'proven_reserves'
    df_clean = df_clean.rename(columns={'Country': 'country', 'ISO3166_alpha3': 'country_code3', 'Year': 'year', 'Value': 'value', 'Var': 'energy_source', 'Region': 'region', 'SubRegion': 'subregion', 'OPEC': 'opec', 'EU' : 'eu', 'OECD': 'oecd', 'CIS': 'cis'})

    # convert unit value
    df_clean.loc[df_clean.unit == 'tcm', 'value'] *= 1000
    df_clean['value'] = df_clean['value'].apply(lambda x : round(x,2))
    df_clean.loc[df_clean['unit']=='tcm', 'unit'] = 'bcm'
    
    df_clean = df_clean[(df_clean.country.str.startswith(('Total', 'Other')) == False)].reset_index(drop=True) 
    
    (item['unit'], item['energy_type'], item['target'], item['group'])
    
    df_clean['unit_id'] = df_clean.apply(lambda x: unit_id_mapping[(x['unit'], x['energy_source'], x['target'])], axis=1)
    df_clean.drop(columns=['unit'], axis=1, inplace=True)
    
    df_country = df_clean[['country', 'country_code3', 'region', 'subregion', 'opec', 'eu', 'oecd', 'cis']]

    df_clean = df_clean.drop(columns=['country_code3', 'region', 'subregion', 'opec', 'eu', 'oecd', 'cis'], axis=1)

    return df_clean, df_country

In [22]:
data_V2, data_V2_Country = clean_data_2(df, unit_id_mapping)

In [23]:
data_V2.head()

,country,year,energy_source,value,target,unit_id
0,Algeria,1980,Gas,3581.46,proven_reserves,0
1,Algeria,1981,Gas,3540.07,proven_reserves,0
2,Algeria,1982,Gas,3477.51,proven_reserves,0
3,Algeria,1983,Gas,3398.59,proven_reserves,0
4,Algeria,1984,Gas,3312.92,proven_reserves,0


In [24]:
data_V2_Country.head() # TO clean and transform into country dimension table

,country,country_code3,region,subregion,opec,eu,oecd,cis
0,Algeria,DZA,Africa,Northern Africa,1.0,0.0,0.0,0.0
1,Algeria,DZA,Africa,Northern Africa,1.0,0.0,0.0,0.0
2,Algeria,DZA,Africa,Northern Africa,1.0,0.0,0.0,0.0
3,Algeria,DZA,Africa,Northern Africa,1.0,0.0,0.0,0.0
4,Algeria,DZA,Africa,Northern Africa,1.0,0.0,0.0,0.0
